In [2]:
!pip install yolov5

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 7.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 953.4/953.4 kB 51.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.3/80.3 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.6/113.6 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.2/62.2 MB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.1/877.1 kB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 84.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.6/82.6 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 4.6 MB/s eta 0:00:00
  Created wheel for fire: filename=fire-0.7.0-py3-none-any.whl size=114249 sha256=39a

In [1]:
import torch
import cv2
import os
from yolov5 import detect

# .venv\scripts\activate.ps1

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [2]:
# 下载预训练的 YOLOv5 模型 (如 yolov5s)
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or 'yolov5m', 'yolov5l', 'yolov5x'

Downloading: "https://github.com/ultralytics/yolov5/zipball/master" to /root/.cache/torch/hub/master.zip
YOLOv5 🚀 2024-10-24 Python-3.10.12 torch-2.5.0+cu121 CPU

100%|██████████| 14.1M/14.1M [00:00<00:00, 300MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs
Adding AutoShape... 


In [3]:
# Colab 連結 google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [11]:
# copy google driver 檔案 到 colab content 資料夾
# copy Gdrive module 到 colab root(content)
# !cp /content/drive/MyDrive/Colab/my_model.h5 /content

# copy Gdrive module 到 colab root(content)
!cp -r /content/drive/MyDrive/Colab/smile_pic /content #-r is used to copy folders recursively.

In [12]:
# 定义图片路径列表
img_paths = ['smile1.jpg', 'smile2.jpg', 'smile3.jpg','smile4.jpg', 'smile5.jpg', 'smile6.jpg', 'smile7.jpg']  # 替换为你的图片路径

In [17]:
# 遍历每张图片
for img_path in img_paths:

    # Construct the full path to the image
    full_img_path = os.path.join('/content/smile_pic', img_path)

    # img = cv2.imread(img_path)

    img = cv2.imread(full_img_path)

    # Check if image was loaded successfully
    if img is None:
        print(f"Error: Could not read image file: {full_img_path}")
        continue  # Skip to the next image

    # 推理 (检测)

    results = model(img)


    # 处理结果
    labels, coords = results.xyxyn[0][:, -1].numpy(), results.xyxyn[0][:, :-1].numpy()

    # 初始化一个标志
    found_smile = False

    # 遍历检测结果
    for label, coord in zip(labels, coords):
        if label == 0:  # 0 是 'person' 类别
            # 可以进一步进行表情检测，例如通过 OpenCV 的 haarcascades 微笑检测
            x1, y1, x2, y2 = int(coord[0] * img.shape[1]), int(coord[1] * img.shape[0]), int(coord[2] * img.shape[1]), int(coord[3] * img.shape[0])
            face = img[y1:y2, x1:x2]

            # 使用 OpenCV 的微笑检测
            gray_face = cv2.cvtColor(face, cv2.COLOR_BGR2GRAY)
            smile_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_smile.xml')
            smiles = smile_cascade.detectMultiScale(gray_face, scaleFactor=1.7, minNeighbors=22, minSize=(25, 25))

            if len(smiles) > 0:
                found_smile = True
                break

    # 输出结果并修改文件名
    if found_smile:
        new_name = os.path.splitext(img_path)[0] + "_有笑臉" + os.path.splitext(img_path)[1]
        os.rename(full_img_path, new_name)
        print(f"图片中存在笑脸，文件名已修改为: {new_name}")
    else:
        print(f"图片中没有笑脸: {img_path}")

/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


图片中没有笑脸: smile1.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


图片中没有笑脸: smile2.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


...

图片中存在笑脸，文件名已修改为: smile5_有笑臉.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


图片中没有笑脸: smile6.jpg


/root/.cache/torch/hub/ultralytics_yolov5_master/models/common.py:892: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with amp.autocast(autocast):


图片中存在笑脸，文件名已修改为: smile7_有笑臉.jpg
